### Example IMC analysis with Morpheus

##### Step 0: import required packages

In [1]:
import morpheus as mp
from morpheus.train import train_classifier

# reload magic
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'morpheus.train'

##### Step 1: create a dataset object

In [2]:
BASE_DIR = "/groups/mthomson/zwang2/IMC/output/cedarsLiver_sz48_pxl3_nc44/temp"
livertumor = mp.SpatialDataset(data_path=f"{BASE_DIR}/crc.h5")

##### Step 2: generate data splits to prepare for model training

Next, we will need to generate train, validation, and test data splits for model training. We want to stratify our splits by the label we want to predict.

In [21]:
livertumor.generate_data_splits(stratify_by='Tcytotoxic', train_size=0.7, test_size=0.15, val_size=0.15)

##### Step 3: train PyTorch model

In [ ]:
train_classifier(livertumor, model_arch="unet", save_model_dir=)

In [ ]:
metadata = livertumor.metadata[['PatientID', 'ImageNumber']]
label = livertumor.label

# merge metadata and label using ImageNumber as the common column
metadata = metadata.merge(label, on='ImageNumber')
metadata = metadata.reset_index().rename(columns={'index': 'patch_index'})

# add misc to metadata with columns "location_x_index" and "location_y_index"
metadata = metadata.join(pd.DataFrame(misc, columns=['location_x_index', 'location_y_index']))

metadata.to_csv('/groups/mthomson/zwang2/IMC/output/cedarsLiver_sz48_pxl3_nc44/temp/metadata.csv', index=False)

In [ ]:
import h5py
with h5py.File('/groups/mthomson/zwang2/IMC/output/cedarsLiver_sz48_pxl3_nc44/temp/crc.h5', 'w') as f:
    # Create a dataset to store the images
    dset = f.create_dataset("images", data=livertumor.intensity, compression="gzip", 
                            chunks=(100, 16, 16, 44), dtype=livertumor.intensity.dtype)
    
    # Create a group to store the metadata
    metadata_numpy = livertumor.metadata.to_records(index=False)
    dset_metadata = f.create_dataset('metadata', data=metadata_numpy, dtype=metadata_numpy.dtype)

    # Create a group to store the channel names
    dset_channel_names = f.create_dataset('channel_names', data=livertumor.channel)